In [2]:
import pandas as pd
import glob
import json
from concurrent.futures import ThreadPoolExecutor

In [ ]:
df = pd.read_csv('data/output_0_500000_b3.csv')


In [6]:
def read_csv_fast(filepath):
    return pd.read_csv(filepath)

# ⚡ Step 3: Read in parallel using threads (I/O bound)
with ThreadPoolExecutor() as executor:
    dfs = list(executor.map(read_csv_fast, files))

# 📎 Step 4: Concatenate into one DataFrame
df = pd.concat(dfs, ignore_index=True)

df

,item_id,human_face,classes
0,27807483,1,"{'laptop': {'count': 1, 'highest_conf_scr': 0...."
1,64799561,0,{}
2,35310338,0,"{'cat': {'count': 1, 'highest_conf_scr': 0.949..."
3,96533681,0,"{'bowl': {'count': 1, 'highest_conf_scr': 0.90..."
4,12144447,1,"{'person': {'count': 1, 'highest_conf_scr': 0...."
...,...,...,...
4998651,70295820,0,"{'bowl': {'count': 1, 'highest_conf_scr': 0.98..."
4998652,87986014,0,"{'train': {'count': 2, 'highest_conf_scr': 0.9..."
4998653,89316176,0,{}
4998654,47500079,1,"{'person': {'count': 1, 'highest_conf_scr': 0...."


In [ ]:
df['classes'] = df['classes'].str.replace("'", '"')
df['classes'] = df['classes'].apply(json.loads)
df

,item_id,human_face,classes
0,73235481,1,"{'person': {'count': 1, 'highest_conf_scr': 0...."
1,71841194,0,"{'dog': {'count': 1, 'highest_conf_scr': 0.875..."
2,72516432,1,"{'person': {'count': 1, 'highest_conf_scr': 0...."
3,52989418,0,"{'person': {'count': 7, 'highest_conf_scr': 0...."
4,82433509,0,"{'bowl': {'count': 1, 'highest_conf_scr': 0.88..."
...,...,...,...
499931,31671660,0,"{'wine glass': {'count': 1, 'highest_conf_scr'..."
499932,15756317,1,"{'person': {'count': 1, 'highest_conf_scr': 0...."
499933,63239721,0,"{'cat': {'count': 1, 'highest_conf_scr': 0.693..."
499934,108295350,0,"{'person': {'count': 1, 'highest_conf_scr': 0...."


In [5]:
# df_classes_fl = df_sorted['classes'].apply(extract_features_fast).apply(pd.Series)
# df_classes_fl

# Flatten nested structure
normalized = pd.json_normalize(df['classes'])
normalized


,person.count,person.highest_conf_scr,dog.count,dog.highest_conf_scr,potted plant.count,potted plant.highest_conf_scr,clock.count,clock.highest_conf_scr,car.count,car.highest_conf_scr,...,baseball bat.count,baseball bat.highest_conf_scr,toaster.count,toaster.highest_conf_scr,hair drier.count,hair drier.highest_conf_scr,skis.count,skis.highest_conf_scr,baseball glove.count,baseball glove.highest_conf_scr
0,1.0,0.940354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,1.0,0.875807,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,0.956479,NaN,NaN,2.0,0.943818,1.0,0.613947,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.0,0.867281,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.841887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499932,1.0,0.974109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499934,1.0,0.605659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_final = pd.concat([df.drop(columns='classes'), normalized], axis=1)

df_final


,item_id,human_face,person.count,person.highest_conf_scr,dog.count,dog.highest_conf_scr,potted plant.count,potted plant.highest_conf_scr,clock.count,clock.highest_conf_scr,...,baseball bat.count,baseball bat.highest_conf_scr,toaster.count,toaster.highest_conf_scr,hair drier.count,hair drier.highest_conf_scr,skis.count,skis.highest_conf_scr,baseball glove.count,baseball glove.highest_conf_scr
0,73235481,1,1.0,0.940354,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,71841194,0,NaN,NaN,1.0,0.875807,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72516432,1,1.0,0.956479,NaN,NaN,2.0,0.943818,1.0,0.613947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52989418,0,7.0,0.867281,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,82433509,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499931,31671660,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499932,15756317,1,1.0,0.974109,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499933,63239721,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499934,108295350,0,1.0,0.605659,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
count_columns = [col for col in df_final.columns if col.endswith('.count')]

df_final['total_count'] = df_final[count_columns].sum(axis=1)
#2s - vectorized
df_final


,item_id,human_face,person.count,person.highest_conf_scr,dog.count,dog.highest_conf_scr,potted plant.count,potted plant.highest_conf_scr,clock.count,clock.highest_conf_scr,...,baseball bat.highest_conf_scr,toaster.count,toaster.highest_conf_scr,hair drier.count,hair drier.highest_conf_scr,skis.count,skis.highest_conf_scr,baseball glove.count,baseball glove.highest_conf_scr,total_count
0,73235481,1,1.0,0.940354,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,71841194,0,NaN,NaN,1.0,0.875807,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,72516432,1,1.0,0.956479,NaN,NaN,2.0,0.943818,1.0,0.613947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,52989418,0,7.0,0.867281,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
4,82433509,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499931,31671660,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
499932,15756317,1,1.0,0.974109,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
499933,63239721,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
499934,108295350,0,1.0,0.605659,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
count_cols = [col for col in df_final.columns if col.endswith('.count')]
df_final['total_count'] = df_final[count_cols].apply(lambda row: row.sum(), axis=1)
# 12s - apply
df_final

,item_id,human_face,person.count,person.highest_conf_scr,dog.count,dog.highest_conf_scr,potted plant.count,potted plant.highest_conf_scr,clock.count,clock.highest_conf_scr,...,baseball bat.highest_conf_scr,toaster.count,toaster.highest_conf_scr,hair drier.count,hair drier.highest_conf_scr,skis.count,skis.highest_conf_scr,baseball glove.count,baseball glove.highest_conf_scr,total_count
0,73235481,1,1.0,0.940354,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,71841194,0,NaN,NaN,1.0,0.875807,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,72516432,1,1.0,0.956479,NaN,NaN,2.0,0.943818,1.0,0.613947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,52989418,0,7.0,0.867281,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
4,82433509,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499931,31671660,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
499932,15756317,1,1.0,0.974109,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
499933,63239721,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
499934,108295350,0,1.0,0.605659,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
